In [20]:
! pip install pynput

  Using cached pynput-1.8.1-py2.py3-none-any.whl.metadata (32 kB)
Using cached pynput-1.8.1-py2.py3-none-any.whl (91 kB)


In [1]:
import sys
print(sys.executable)

c:\Users\Pc\anaconda3\envs\env_detect_drone\python.exe


In [2]:
import numpy as np
import cv2
import threading
import queue
import torch
import time
from pynput import keyboard
from ultralytics import YOLO

Тестирорвание сохранения видео из одной камеры как с предсказанием детектирования объектов так и без них. Потому что скорость видео разная, нужно найти почему

In [3]:
# model = YOLO('yolo11m.pt')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

export model

In [4]:
model = YOLO('yolo11m.pt') # or YOLO(path/to/best.pt)

#export ncnn
# model.export(format='ncnn',
#              int8=False,
#              dynamic=False,
#              half = True,
#              imgsz = 1024,
#              device=device,
#              batch = 1
#              )

# export onnx
model.export(format='onnx',
            #  imgsz=640,
             half=True,
             dynamic=True,
             nms = True,
             device=device)

WARNING 'dynamic=True' model with 'nms=True' requires max batch size, i.e. 'batch=16'
YOLO11m summary (fused): 125 layers, 20,091,712 parameters, 0 gradients, 68.0 GFLOPs

PyTorch: starting from 'yolo11m.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 300, 6) (38.8 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.59', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ---------------------------------------- 14.5/14.5 MB 7.9 MB/s eta 0:00:00
   ---------------------------------------- 214.9/214.9 MB 7.4 MB/s eta 0:00:00
   ---------------------------------------- 7/7 [onnxruntime-gpu]

requirements: AutoUpdate success  50.9s
WARNING requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.65...
ONNX: export success  60.0s, saved as 'yolo11m.onnx' (38.6 MB)

Export complete (65.0s)
Results saved to C:\TASK_DETECT_DRONE\CODES_D

'yolo11m.onnx'

In [11]:
# load model ncnn format
ncnn_model = YOLO('yolo11m_ncnn_model')

# load model onnx format
# onnx_model = YOLO('yolo11m.onnx')

WARNING Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [ ]:
def capture_camera_process(idx_cam: int):
    """function to work in diferent process - capture frame and sent in queue"""
    st = time.time()
    cap = cv2.VideoCapture(idx_cam)#, cv2.CAP_DSHOW
    st1 = time.time() - st
    if not cap.isOpened():
        print(f'Cam {idx_cam} not opened')
        return
    st2 = time.time() - st
    #set params for cameras fps, width, height
    cap.set(cv2.CAP_PROP_FPS, 30)
    st3  = time.time() - st
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    st4 = time.time() - st
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    st5 = time.time() - st


    fps = int(cap.get(cv2.CAP_PROP_FPS))
    st6 = time.time() - st
    # if fps == 0:
    #     fps = 30
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    st7 = time.time() - st
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    st8 = time.time() - st
    print(f"[Процесс {idx_cam}] Подключено: {width}x{height} @ {fps}fps")
    start_time = time.time()
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    current_time = time.strftime("%Y%m%d_%H%M%S")
    filename = fr'record_cam1\video_{current_time}.avi'
    writer = cv2.VideoWriter(filename, fourcc, fps, (width, height))
    print(f'Prepaire writer: {time.time() - start_time} second')

    count_cadr = 0
    start_time = time.time()
    print('loop while...')
    while True:
        ret, frame = cap.read()
        if not ret:
            print(f'Not get frame')
            break 

        result = ncnn_model(frame, conf=0.5, iou=0.2, imgsz=1024, verbose=True, device=device)
        
        cls = result[0].boxes.cls.cpu().numpy()
        coords = result[0].boxes.xyxy.cpu().numpy()
        for cl, (x, y, x1, y1) in zip(cls, coords):
            cv2.rectangle(frame, (int(x), int(y)), (int(x1), int(y1)), (255, 0, 0), 1)
            # Добавляем текст на кадре (можно и в главном процессе, но так легче)
            
            text_cls = str(cl)
            cv2.putText(frame, text_cls, (int(x), int(y-3)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0),1)

        text = f'Cam {idx_cam} | {width}x{height} @ {fps}fps'
        cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        # if queue is full
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print('Exit by command user')
            break
        count_cadr += 1
        delta_time = time.time() -start_time
        print(f'Curent time: {delta_time}', end='\r', flush=True)
        cv2.imshow(f'video_time', frame)

        writer.write(frame)


    cap.release()
    writer.release()
    cv2.destroyAllWindows()
    end_time = time.time() - start_time
    # print(f'st - {st}, \nst1 - {st1}, \nst2 - {st2}, \nst3 - {st3}, \nst4 - {st4}, \nst5 - {st5}, \nst6 - {st6}, \nst7 - {st7}, \nst8 - {st8}')
    print(f'Process cam {idx_cam} END. \nTime: {end_time} \nCount cadr: {count_cadr}')
    real_fps = count_cadr / end_time
    print(f'Real fps: {real_fps}')



capture_camera_process(1)

No predict:
[Процесс 0] Подключено: 1280x720 @ 30fps
Prepaire writer: 0.0010001659393310547 second
Exit by command user32980346686
Process cam 0 END. 
Time: 30.57908797264099 
Count cadr: 890
Real fps: 29.104857567899998


C predict:1024
[Процесс 0] Подключено: 1280x720 @ 30fps
Prepaire writer: 0.0030198097229003906 second
Exit by command user19241333008
Process cam 0 END. 
Time: 30.683942317962646 
Count cadr: 423
Real fps: 13.785712266587469


predict 640
[Процесс 1] Подключено: 640x480 @ 30fps
Prepaire writer: 0.001001596450805664 second
loop while...
Exit by command user66066360474
Process cam 1 END. 
Time: 30.737857580184937 
Count cadr: 461
Real fps: 14.997792178501802

predict 224
[Процесс 1] Подключено: 640x480 @ 30fps
Prepaire writer: 0.0020008087158203125 second
loop while...
Exit by command user98002243042
Process cam 1 END. 
Time: 30.767577648162842 
Count cadr: 449
Real fps: 14.59328404512242

predict 1024 onnx export
[Процесс 1] Подключено: 1280x720 @ 30fps
Prepaire writer: 0.004999876022338867 second
loop while...
Loading yolo11m.onnx for ONNX Runtime inference...
Using ONNX Runtime CUDAExecutionProvider
Exit by command user67093658447
Process cam 1 END. 
Time: 30.84447145462036 
Count cadr: 334
Real fps: 10.828520776937104

predict 640 onnx export
[Процесс 1] Подключено: 1280x720 @ 30fps
Prepaire writer: 0.0019998550415039062 second
loop while...
Exit by command user97754287723
Process cam 1 END. 
Time: 30.860729694366455 
Count cadr: 526
Real fps: 17.04431506349054


Set 23 fps imgsz=640 format=onnx
[Процесс 1] Подключено: 1280x720 @ 25fps
Prepaire writer: 0.0010023117065429688 second
loop while...
Exit by command user97525787354
Process cam 1 END. 
Time: 30.917776584625244 
Count cadr: 526
Real fps: 17.012866321751243



1024
image size = 1024 predict 52 - 55ms frame
7.9ms preprocess, 34.5ms inference, 5.8ms postprocess per image at shape (1, 3, 768, 1024)
Curent time: 0.9589450359344482
0: 768x1024 1 person, 43.8ms
Speed: 8.4ms preprocess, 43.8ms inference, 4.1ms postprocess per image at shape (1, 3, 768, 1024)
Curent time: 1.051941156387329
0: 768x1024 1 person, 1 remote, 37.6ms
Speed: 11.3ms preprocess, 37.6ms inference, 3.7ms postprocess 

640
predict ~ 48 ms
Speed: 2.0ms preprocess, 32.4ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 640)
Curent time: 0.8921859264373779
0: 480x640 1 person, 1 remote, 32.0ms
Speed: 2.2ms preprocess, 32.0ms inference, 3.9ms postprocess per image at shape (1, 3, 480, 640)
Curent time: 0.9621872901916504
0: 480x640 1 person, 1 remote, 40.9ms
Speed: 2.2ms preprocess, 40.9ms inference, 5.2ms postprocess per image at shape (1, 3, 480, 640)
Curent time: 1.0502090454101562
0: 480x640 1 person, 1 remote, 39.9ms
Speed: 3.4ms preprocess, 39.9ms inference, 3.8ms postprocess per image at shape (1, 3, 480, 640)
Curent time: 1.1407248973846436



640 onnx format
predict ~ 31 ms
Speed: 1.9ms preprocess, 30.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Curent time: 0.8115992546081543
0: 384x640 1 person, 28.5ms
Speed: 1.9ms preprocess, 28.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Curent time: 0.8685996532440186
0: 384x640 1 person, 26.8ms
Speed: 1.9ms preprocess, 26.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Curent time: 0.9225924015045166
0: 384x640 1 person, 25.9ms
Speed: 1.9ms preprocess, 25.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Curent time: 0.9755916595458984
0: 384x640 1 person, 30.0ms
Speed: 1.9ms preprocess, 30.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Curent time: 1.0335912704467773



1024 ncnn format
predict ~ 1370
Speed: 14.0ms preprocess, 1338.9ms inference, 4.2ms postprocess per image at shape (1, 3, 1024, 1024)
Curent time: 5.110432863235474
0: 1024x1024 1 person, 1324.1ms
Speed: 13.0ms preprocess, 1324.1ms inference, 4.2ms postprocess per image at shape (1, 3, 1024, 1024)
Curent time: 6.483858585357666
0: 1024x1024 1 person, 1339.0ms
Speed: 12.3ms preprocess, 1339.0ms inference, 4.6ms postprocess per image at shape (1, 3, 1024, 1024)
Curent time: 7.870999336242676
0: 1024x1024 1 person, 1351.2ms
Speed: 14.2ms preprocess, 1351.2ms inference, 5.2ms postprocess per image at shape (1, 3, 1024, 1024)
Curent time: 9.275911331176758
0: 1024x1024 1 person, 1363.8ms
Speed: 14.4ms preprocess, 1363.8ms inference, 4.6ms postprocess per image at shape (1, 3, 1024, 1024)
...
Process cam 1 END. 
Time: 34.312185764312744 
Count cadr: 23
Real fps: 0.6703157927036444

In [3]:
vid = r'C:\TASK_DETECT_DRONE\CODES_DETECT_DRONE\YOLO+calc_distance\record_cam1\video_20250825_122051.avi'
cap = cv2.VideoCapture(vid)
count_cadrs = cap.get(cv2.CAP_PROP_FRAME_COUNT)
count_cadrs

58.0

In [4]:

! ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate,nb_frames -of default=noprint_wrappers=1:nokey=1 {vid}

30/1
58


тест отдельно по камерам

In [5]:
idx_cam = 0
cap = cv2.VideoCapture(idx_cam)
if not cap.isOpened():
    print(f"Error open camera {idx_cam}")
    # return

fps = int(cap.get(cv2.CAP_PROP_FPS))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
font = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX

# fourcc = cv2.VideoWriter_fourcc(*'XVID')  #MP4V   пропуск кадров
# output_video = rf'{path_save_video}\video_cam_{idx_cam}.avi'  # пропуск кадров
# writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))  # пропуск кадров

while True:
    ret, frame = cap.read()
    if not ret:
        print(f'Lost connetct with camera id: {idx_cam}')
        break
    text = f'{str(fps)} : {str(height)} : {str(width)} '
    cv2.putText(frame, text, (10, 30), font, 1.0, (255, 0, 0), 1)

    # writer.write(frame)  # пропуск кадров

    # clearing the queue (чтоб не тормозила камера) so that the camera doesn't slow down
    

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Was pressed key quit")
        break

    cv2.imshow(f'camera_{str(idx_cam)}', frame)

cap.release()
cv2.destroyAllWindows()

Was pressed key quit


define function for visualize from camera

In [2]:
idx_cam_1 = 0
idx_cam_2 = 1

frame_queue1 = queue.Queue(maxsize=1)
frame_queue2 = queue.Queue(maxsize=1)

writers={}

def capture_camera(idx_cam: int, frame_queue):
    cap = cv2.VideoCapture(idx_cam)
    if not cap.isOpened():
        print(f"Error open camera {idx_cam}")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    font = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX

    # fourcc = cv2.VideoWriter_fourcc(*'XVID')  #MP4V   пропуск кадров
    # output_video = rf'{path_save_video}\video_cam_{idx_cam}.avi'  # пропуск кадров
    # writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))  # пропуск кадров
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print(f'Lost connetct with camera id: {idx_cam}')
            break
        text = f'{str(fps)} : {str(height)} : {str(width)} '
        cv2.putText(frame, text, (10, 30), font, 1.0, (255, 0, 0), 1)

        # writer.write(frame)  # пропуск кадров

        # clearing the queue (чтоб не тормозила камера) so that the camera doesn't slow down
        if not frame_queue.empty():
            frame_queue.get()
        
        frame_queue.put((idx_cam, frame))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Was pressed key quit")
            break

        # cv2.imshow(f'camera_{str(idx_cam)}', frame)
    
    cap.release()
    # writer.release()   # пропуск кадров
    # cv2.destroyAllWindows()


### Example with threading

In [3]:
# start threading
idx_cam_1 = 0
idx_cam_2 = 1

frame_queue1 = queue.Queue(maxsize=1)
frame_queue2 = queue.Queue(maxsize=1)

writers={}

def capture_camera(idx_cam: int, frame_queue):
    cap = cv2.VideoCapture(idx_cam)
    if not cap.isOpened():
        print(f"Error open camera {idx_cam}")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    font = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX

    # fourcc = cv2.VideoWriter_fourcc(*'XVID')  #MP4V   пропуск кадров
    # output_video = rf'{path_save_video}\video_cam_{idx_cam}.avi'  # пропуск кадров
    # writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))  # пропуск кадров
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print(f'Lost connetct with camera id: {idx_cam}')
            break
        text = f'{str(fps)} : {str(height)} : {str(width)} '
        cv2.putText(frame, text, (10, 30), font, 1.0, (255, 0, 0), 1)

        # writer.write(frame)  # пропуск кадров

        # clearing the queue (чтоб не тормозила камера) so that the camera doesn't slow down
        if not frame_queue.empty():
            frame_queue.get()
        
        frame_queue.put((idx_cam, frame))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Was pressed key quit")
            break

        # cv2.imshow(f'camera_{str(idx_cam)}', frame)
    
    cap.release()
    # writer.release()   # пропуск кадров
    # cv2.destroyAllWindows()



def main():

    path_save1 = r'record_cam1'
    path_save2 = r'record_cam2'

    t1 = threading.Thread(target=capture_camera, args=(idx_cam_1, frame_queue1), daemon=True)
    t2 = threading.Thread(target=capture_camera, args=(idx_cam_2, frame_queue2), daemon=True)

    t1.start()
    t2.start()

    # Main loop visualize
    try:
        while True:
            current_time = time.strftime("%Y%m%d_%H%M%S")
            

            if not frame_queue1.empty():
                idx, frame = frame_queue1.get()
                cv2.imshow(f'Camera_{idx}', frame)

                #Initialize VideoWriter 
                if idx not in writers:
                    filename = fr'{path_save1}\camera_{idx}_{current_time}.mp4'
                    fourcc = cv2.VideoWriter_fourcc(*'H264')  # XVID
                    writers[idx] = cv2.VideoWriter(filename, fourcc, 30, (frame.shape[1], frame.shape[0]))
                    print(f'Record start: {filename}')
                
                writers[idx].write(frame)
            
            if not frame_queue2.empty():
                idx, frame = frame_queue2.get()
                cv2.imshow(f'Camera_{idx}', frame)

                #init vwr
                if idx not in writers:
                    filename = fr'{path_save2}\camera_{idx}_{current_time}.avi'
                    fourcc = cv2.VideoWriter_fourcc(*'XVID')
                    writers[idx] = cv2.VideoWriter(filename, fourcc, 30, (frame.shape[1], frame.shape[0]))
                    print(f'Record avi start: {filename}')
                
                writers[idx].write(frame)

            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            time.sleep(0.01) 
    except KeyboardInterrupt:
        pass
    for writer in writers.values():
        writer.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


Record start: record_cam1\camera_0_20250724_143334.mp4
Record avi start: record_cam2\camera_1_20250724_143334.avi


Если камеры подключены по сети (IP-камеры), то cv2.VideoCapture(idx_cam) может работать нестабильно — лучше использовать cv2.CAP_FFMPEG или cv2.CAP_DSHOW в зависимости от платформы.

### use multiprocessing

In [4]:
import cv2
import multiprocessing as mp
import time



In [2]:
def capture_camera_process(idx_cam: int, queue: mp.Queue):
    """function to work in diferent process - capture frame and sent in queue"""
    cap = cv2.VideoCapture(idx_cam)
    if not cap.isOpened():
        print(f'Cam {idx_cam} not opened')
        return
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"[Процесс {idx_cam}] Подключено: {width}x{height} @ {fps}fps")


    while True:
        ret, frame = cap.read()
        if not ret:
            print(f'Not get frame')
            break
        # Добавляем текст на кадре (можно и в главном процессе, но так легче)
        text = f'Cam {idx_cam} | {width}x{height} @ {fps}fps'
        cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        # if queue is full
        if not queue.empty():
            queue.get() # cleareing queue
        queue.put((idx_cam, frame))

        
        time.sleep(0.01)
    cap.release()
    queue.put((idx_cam, None)) # Signal to end
    print(f'Process cam {idx_cam} END.')

In [3]:
mp.set_start_method('spawn', force=True)  # Критично для Windows/macOS
idx_cam_1 = 0
idx_cam_2 = 1

# Создаём очереди для передачи кадров из процессов
queue1 = mp.Queue(maxsize=1)
queue2 = mp.Queue(maxsize=1)

# Запускаем процессы
proc1 = mp.Process(target=capture_camera_process, args=(idx_cam_1, queue1), daemon=True)
proc2 = mp.Process(target=capture_camera_process, args=(idx_cam_2, queue2), daemon=True)

proc1.start()
proc2.start()

print('Start processes to capture cadrs. Press "q" to exit')
try:
    while True:
        frame1 = None
        frame2 = None

        if not queue1.empty():
            cam_id, frame1 = queue1.get()
            
            if frame1 is None:
                print('Camera 1 is stop')
                break
        
        if not queue2.empty():
            cam_id, frame2 = queue2.get()
            if frame2 is None:
                print('Camera 2 is stop')
                break

        if frame1 is not None:
            cv2.imshow(f'Camera_{cam_id}', frame1)
        if frame2 is not None:
            cv2.imshow(f'Camera {cam_id}', frame2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print('Exit by command user')
            break
        time.sleep(0.01)
except KeyboardInterrupt:
    print('Stopped USER')

finally:
    # stoped processes
    cv2.destroyAllWindows()
    proc1.terminate()
    proc2.terminate()
    proc1.join(timeout=1)
    proc2.join(timeout=1)
    cv2.destroyAllWindows()
    print('Resourses to free')


Start processes to capture cadrs. Press "q" to exit
Stopped USER
Resourses to free


In [4]:
print('Hello')

Hello


Пример изменения источника камеры по нажатию клавиши, НЕ ТАК КАК НАДО работает, просто пример

In [ ]:

cam = "camera_1"

def on_press(key):
    global cam
    try:
        if key.char == ' ': #
            if cam == 'camera_1':
                cam == 'camera_2'
                get_video(idx_cam_2)
            else:
                get_video(idx_cam_1)
    except Exception as e:
        print(f'Error: {e}')

listener = keyboard.Listener(on_press=on_press)
listener.start()

print(f'Press space for change camera. Ctrl+C for exit.')

try:
    while True:
        if cam == 'camera_1':
            print(f'Release camera 1')
            get_video(idx_cam_1)
        else:
            print(f'Release camera 2')
            get_video(idx_cam_2)
except KeyboardInterrupt:
    listener.stop()
    print("\nExit")

Press space for change camera. Ctrl+C for exit.
Release camera 1
Press quit


Если камеры подключены по сети (IP-камеры), то cv2.VideoCapture(idx_cam) может работать нестабильно — лучше использовать cv2.CAP_FFMPEG или cv2.CAP_DSHOW в зависимости от платформы.

Read count frames in video

In [5]:
path = r'C:\TASK_DETECT_DRONE\CODES_DETECT_DRONE\YOLO+calc_distance\record_cam1\camera_0_20250825_104555.avi'
! ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate,nb_frames -of default=noprint_wrappers=1:nokey=1 {path}

30/1
155


In [6]:
path = r'C:\TASK_DETECT_DRONE\CODES_DETECT_DRONE\YOLO+calc_distance\record_cam1\camera_0_20250825_112356.avi'
! ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate,nb_frames -of default=noprint_wrappers=1:nokey=1 {path}

30/1
78


In [4]:
import cv2
import time

path = r'C:\TASK_DETECT_DRONE\CODES_DETECT_DRONE\YOLO+calc_distance\record_cam1\camera_0_20250825_104555.avi'
def describe_video(path):
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        print('cap is not opened')

    count_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration_sec = count_frames / fps
    print(f'fps = {fps}, \nduration video in second:  {duration_sec}')

describe_video(path)



fps = 30.0, 
duration video in second:  5.166666666666667
